# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, recall_score,precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>

In [ ]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

### 1 регион

In [ ]:
data0.head()

In [ ]:
data0.info()

In [ ]:
data0.describe()

In [ ]:
data0.duplicated().sum()

### 2 Регион

In [ ]:
data1.head()

In [ ]:
data1.info()

In [ ]:
data1.describe()

In [ ]:
data0.duplicated().sum()

### 3 регион

In [ ]:
data2.head()

In [ ]:
data2.info()

In [ ]:
data2.describe()

In [ ]:
data2.duplicated().sum()

In [ ]:
#  удалю столбцы с айди
data0.drop(['id'], axis=1, inplace=True)
#print(data0.head())
data1.drop(['id'], axis=1, inplace=True)
#print(data1.head())
data2.drop(['id'], axis=1, inplace=True)
#print(data2.head())

### Вывод
Данные в порядке, пропусков нет

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены, но можно было бы еще как минимум изучить корреляции, так как мы планируем использовать линейные модели, а для них это важно. 
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b>  Советую обратить внимание на библиотеки sweetviz и pandas_profiling помогут в проведении более тщательного EDA анализа. Исследовательский анализ можно делать и с помощью ручного вызова функций дефолтных библиотек. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям.  

Очень просты в использоовании, на вход кладется датафрейм: pandas_profiling.ProfileReport(df) 
</div>


## Обучение и проверка модели


Обучите и проверьте модель для каждого региона:

 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
 
 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
 
 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
 
 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
 
 2.5. Проанализируйте результаты.

In [ ]:
# создам функцию для обучения модели
def split_and_train(data):
    
    features = data.drop(["product"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=1)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression(normalize=False)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    mean = predictions.mean()
    
    print("RMSE:{0:.2f}".format(rmse))
    print("Mean: {0:.2f}".format(mean))
    
    return (predictions, target_valid.reset_index(drop=True), rmse)

In [ ]:
#ну и расчет 
i = 1
for region in [data0,data1,data2]:
    print(f"Регион {i}")
    split_and_train(region)
    i+=1

In [ ]:
#  predictions, true values и rmse
predictions0, validation0, rmse0 = split_and_train(data0)
predictions1, validation1, rmse1 = split_and_train(data1)
predictions2, validation2, rmse2 = split_and_train(data2)

### Вывод
Модели обучены и проверены на качество предсказания метрикой RMSE, которая определяет величину среднеквадратичной ошибки в предсказании модели. Исходя из полученных данных, можно заметить, что величина RMSE во втором регионе сильно отличается от двух других.

<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан лаконично, все метрики посчитаны корректно
</div> 




## Подготовка к расчёту прибыли

Подготовьтесь к расчёту прибыли:

 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
 
 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 
 
 3.3. Напишите выводы по этапу подготовки расчёта прибыли.

При разведке региона исследуюем 500 точек, из которых с помощью машинного обучения выберем 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс.
рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью.

In [ ]:
# создаю переменные
well = 500 # При разведке региона исследуют 500 точек
well_model = 200  #с помощью машинного обучения выбирают 200 точек лучших для разработки
budget = 10e9 # Бюджет на разработку 200 скважин в регионе
barrel_profit = 450 # доход с каждого барреля
product_profit = barrel_profit * 1000 # доход с каждой единицы продукта
risk = 0.025 # порог вероятности убытков, максимальное допустимое значение
# мин объем сырья
necessary_product = budget / well_model / product_profit
print('Необходимый объём для окупаемости заложенного бюджета:', necessary_product)


<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    

</div> 


In [ ]:
#запасы
product_mean0 = data0['product'].mean()
product_mean1 = data1['product'].mean()
product_mean2 = data2['product'].mean()


In [ ]:
print('Cредний запас сырья в 1 регионе: %.2f '%product_mean0)
print('Cредний запас сырья в 2 регионе: %.2f '%product_mean1)
print('Cредний запас сырья в 3 регионе: %.2f '%product_mean2)

In [ ]:
# отклонение от среднего в регионе 1 
mean0 = product_mean0 - necessary_product
mean0*1000

In [ ]:
# отклонение от среднего в регионе 2 

mean1 = product_mean1 - necessary_product
mean1*1000

In [ ]:
# отклонение от среднего в регионе 3

mean2 = product_mean2 - necessary_product
mean2*1000

### Вывод

В каждом из трёх регионов средни запас сырья скважины меньше полученного значения достаточного объёма сырья на 18.5, 42.3, 16.1 тыс. баррелей. На основании данного заключения можно отметить высокий риск разработок скважин преимущественно в первом регионе

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>

## Расчёт прибыли и рисков 

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

 4.1. Выберите скважины с максимальными значениями предсказаний. 
 
 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
 
 4.3. Рассчитайте прибыль для полученного объёма сырья.

In [ ]:
#predictions0

In [ ]:
#sorted0 = pd.Series(predictions0, index=validation0.index).sort_values(ascending=False)
#max0 = sorted0[:200]
#max0

In [ ]:
#max0_sum = max0.sum()
#print('Суммарный запас сырья в 200 лучших скважинах составит: %.2f тыс.баррелей'%max0_sum)

In [ ]:
#profit0=(max0_sum*barrel_profit)-10000000
#print('Прибыль для полученного объёма сырья составит %.2f тыс.рублей'%profit0)

In [ ]:
#напишу функцию для расчета суммарного запаса
def profit(predict,target,count):
    sorted_p = pd.Series(predict, index=target.index).sort_values(ascending=False)
    s_max = sorted_p[:count].sum()
    profit=(s_max*barrel_profit)-10000000
    print('Суммарный запас сырья в 200 лучших скважинах составит: %.2f тыс.баррелей'%s_max)
    print('Прибыль для полученного объёма сырья составит %.2f млн.рублей'%(profit/1000))

<div class="alert alert-block alert-success">
<b>Успех:</b> Ф-ция для подсчета прибыли определена верно
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b> Что за магические число? 10000000?
</div>

In [ ]:
profit(predictions0,validation0,200)


In [ ]:
profit(predictions1,validation1,200)


In [ ]:
profit(predictions2,validation2,200)


### Вывод

Все три региона потенциально прибыльные, для определения наиболее выгодного с точки зрения окупаемости вложений и рисков убытка проведём расчёт вероятности техникой бутстреп

## Посчитайте риски и прибыль для каждого региона

5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
 
 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.



well = 500 # При разведке региона исследуют 500 точек
well_model = 200  #с помощью машинного обучения выбирают 200 точек лучших для разработки
budget = 10e9 # Бюджет на разработку 200 скважин в регионе
barrel_profit = 450 # доход с каждого барреля
product_profit = barrel_profit * 1000 # доход с каждой единицы продукта
risk = 0.025 # порог вероятности убытков, максимальное допустимое значение


In [ ]:
def profit(target, predictions, well_model):
    p_sorted = predictions.sort_values(ascending=False)
    selected = target[p_sorted.index][:200]
    return (barrel_profit*1000) * selected.sum() - budget


<div class="alert alert-block alert-info">
<b>Совет:</b> Зачем дублировать функцию прибыли?
</div>

In [ ]:
#применяю технику бутстреп
target = [validation0, validation1, validation2]
predictions = [predictions0, predictions1, predictions2]
regions = ['1 регион', '2 регион', '3 регион']

state = np.random.RandomState(12345)

for m in range(len(target)):
    values = []
    for i in range(1000):
        target_subsample = target[m].sample(n=well, replace=True, random_state=state)
        predictions_subsample = predictions[m].loc[target_subsample.index]
        values.append(profit(
            target_subsample.reset_index(drop=True), predictions_subsample.reset_index(drop=True), well_model)
                     )

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)

    mean = values.mean()
    
    risk = sum(values < 0) / values.count() * 100
    
    print(regions[m])
    print("Средняя возможная прибыль:", mean)
    print("95%-доверительный интервал:", "от", lower, "до", upper)
    print("Вероятность убытков:", risk, "%")


<div class="alert alert-block alert-success">
<b>Успех:</b> С бутстрапом тоже все в порядке, метрики считаются верным образом!  
</div>


### Вывод

Вероятность убытков во втором регионе минимальная и возможная средняя прибыль у него самая высокая

## Вывод


Для выполнения поставленной задачи были обработаны данные трёх регионов.


Наиболее точные предсказания оказались у модели второго региона: средний запас сырья в скважине 68.83 при отклонении 0.89, предсказания по первому и третьему региону получились схожими между собой.

Средняя прибыль во втором регионе составит 476 млн.рублей. Два других региона не прошли отбор по минимально допустимому порогу убытка в 2,5%. Таким образом не смотря на сравнительно меньшие запасы сырья, по сравнению с другими регионами, средняя доходность со скважин во втором регионе прогнозируется выше других.



<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, проведено детальное исследование и был выбран правильный регион!
</div>





<div class="alert alert-block alert-info">
<b>Совет: </b> Если интересно изучить тему бутстрапа глубже - я оставлю  ниже несколько интересных и полезных ссылок по теме:
        
1. "Bootstrap Hypothesis Testing in Statistics with Example" 
        
https://www.youtube.com/watch?v=9STZ7MxkNVg
        
2. "How to Calculate Bootstrap Confidence Intervals For Machine Learning Results in Python" 
        
https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
        
3. "A Gentle Introduction to the Bootstrap Method" 

https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
        
4. "An Introduction to the Bootstrap Method" 
        
https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60#:~:text=The%20basic%20idea%20of%20bootstrap,population%20mean)%20on%20sample%20data.&amp;text=A%20sample%20from%20population%20with%20sample%20size%20n
        
5. "Bootstrapping and Resampling in Statistics with Example" 
        
        
https://www.youtube.com/watch?v=O_Fj4q8lgmc
</div>


:)

<div class="alert alert-block alert-success">
<b>Успех:</b> =)
</div>


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован